In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# 전체 체크 항목에 대해서 다 가져오고 싶다 

In [39]:
url = requests.get("https://finance.naver.com/sise/sise_market_sum.naver?sosok=0&page=1")
html = BeautifulSoup(url.text)

In [40]:
def make_table(html):
    table = html.find("table", class_ = "type_2")
    table = pd.read_html(str(table))[0]            # 리스트의 0번쨰 인덱스에 표가 들어있음  =  리스트의 0번째만 가져오면 데이터프레임형태로 표 가져올 수 있음 
    del table["N"]
    del table["토론실"]
    table = table.dropna(subset = "종목명")        # 종목명에 누락값이 있는 행은 다 삭제하겠다
    return table

In [41]:
items = html.find("table", class_ = "item_list").find_all("input", attrs = {"name" : "fieldIds"})

items_box = []
for i in items:
    items_box.append(i["value"]) 
items_box

['quant',
 'ask_buy',
 'amount',
 'market_sum',
 'operating_profit',
 'per',
 'open_val',
 'ask_sell',
 'prev_quant',
 'property_total',
 'operating_profit_increasing_rate',
 'roe',
 'high_val',
 'buy_total',
 'frgn_rate',
 'debt_total',
 'net_income',
 'roa',
 'low_val',
 'sell_total',
 'listed_stock_cnt',
 'sales',
 'eps',
 'pbr',
 'sales_increasing_rate',
 'dividend',
 'reserve_ratio']

In [43]:
dic = {"menu" : "market_sum", 
       "returnUrl" : "http://finance.naver.com/sise/sise_market_sum.naver?sosok=0&page=1",
       "fieldIds" : items_box}
url = requests.post("https://finance.naver.com/sise/field_submit.naver",      # 추가적으로 요청하고 싶은 게 있으니까  post로 보내야함. ?
                 data = dic)
html = BeautifulSoup(url.text)

table = make_table(html)
table.to_excel("시가총액.xlsx", index = False)
table.head()

,종목명,현재가,전일비,등락률,액면가,거래량,거래대금,전일거래량,시가,고가,...,주당순이익,보통주배당금,매출액증가율,영업이익증가율,외국인비율,PER,ROE,ROA,PBR,유보율
1,삼성전자,56500.0,하락 700,-1.22%,100.0,8018191.0,453013.0,14625181.0,57000.0,57100.0,...,4950.0,1446.0,16.20,398.34,50.07,11.41,9.03,7.10,0.97,41772.8
2,SK하이닉스,205500.0,"상승 5,000",+2.49%,5000.0,1365498.0,276319.0,3651613.0,199000.0,206000.0,...,14328.0,1200.0,-26.57,-213.52,55.84,14.34,-15.61,-8.95,2.17,1397.1
3,LG에너지솔루션,372000.0,"하락 1,500",-0.40%,500.0,59187.0,22011.0,140338.0,369500.0,376000.0,...,-1550.0,NaN,31.83,78.23,4.53,-240.00,6.36,3.91,4.31,16691.6
4,삼성바이오로직스,1158000.0,"하락 20,000",-1.70%,2500.0,23996.0,27872.0,108833.0,1181000.0,1187000.0,...,14788.0,NaN,23.10,13.22,13.58,78.31,9.12,5.26,7.78,5432.6
5,현대차,206000.0,"상승 3,500",+1.73%,5000.0,278258.0,57181.0,651155.0,203500.0,207000.0,...,45724.0,11400.0,14.43,53.96,37.18,4.51,13.68,4.56,0.54,6248.8


# 코스피, 코스닥 모두 가져오고 (열도 모든 열)  ->  하나의 데이터 프레임으로 

In [53]:
url = requests.get("https://finance.naver.com/sise/sise_market_sum.naver?sosok=0&page=1")
html = BeautifulSoup(url.text)
kospi_page = int(html.find("td", class_ = "pgRR").find("a")["href"].split("=")[-1])
kospi_page

47

In [55]:
from tqdm import tqdm

total = []
for i in tqdm(range(1, kospi_page+1)):                                          
    dic = {"menu" : "market_sum", 
       "returnUrl" : f"http://finance.naver.com/sise/sise_market_sum.naver?sosok=0&page={i}",
       "fieldIds" : items_box}
    url = requests.post("https://finance.naver.com/sise/field_submit.naver", data = dic)
    html = BeautifulSoup(url.text)
    table = make_table(html)
    total.append(table)
    time.sleep(1 + random.random())

100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [01:30<00:00,  1.92s/it]


In [57]:
kospi = pd.concat(total, ignore_index = True)
kospi["소속"]  = "KOSPI"
kospi

,종목명,현재가,전일비,등락률,액면가,거래량,거래대금,전일거래량,시가,고가,...,보통주배당금,매출액증가율,영업이익증가율,외국인비율,PER,ROE,ROA,PBR,유보율,소속
0,삼성전자,56400.0,하락 800,-1.40%,100.0,8375923.0,473227.0,14625181.0,57000.0,57100.0,...,1446.0,16.20,398.34,50.07,11.39,9.03,7.10,0.97,41772.8,KOSPI
1,SK하이닉스,204500.0,"상승 4,000",+2.00%,5000.0,1475810.0,298985.0,3651613.0,199000.0,206000.0,...,1200.0,-26.57,-213.52,55.84,14.27,-15.61,-8.95,2.16,1397.1,KOSPI
2,LG에너지솔루션,371500.0,"하락 2,000",-0.54%,500.0,59939.0,22291.0,140338.0,369500.0,376000.0,...,NaN,31.83,78.23,4.53,-239.68,6.36,3.91,4.30,16691.6,KOSPI
3,삼성바이오로직스,1159000.0,"하락 19,000",-1.61%,2500.0,24619.0,28593.0,108833.0,1181000.0,1187000.0,...,NaN,23.10,13.22,13.58,78.37,9.12,5.26,7.79,5432.6,KOSPI
4,현대차,206000.0,"상승 3,500",+1.73%,5000.0,289503.0,59493.0,651155.0,203500.0,207000.0,...,11400.0,14.43,53.96,37.18,4.51,13.68,4.56,0.54,6248.8,KOSPI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2305,TIGER 200 산업재,8215.0,하락 55,-0.67%,0.0,1151.0,9.0,17948.0,8270.0,8270.0,...,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,KOSPI
2306,PLUS 코스닥150선물인버스,5440.0,하락 40,-0.73%,0.0,18.0,0.0,10576.0,5485.0,5485.0,...,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,KOSPI
2307,HANARO 글로벌백신치료제MSCI,11800.0,상승 25,+0.21%,0.0,1049.0,12.0,509.0,11795.0,11860.0,...,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,KOSPI
2308,HANARO 미국애그테크,11750.0,상승 40,+0.34%,0.0,806.0,9.0,623.0,11555.0,11750.0,...,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,KOSPI


In [58]:
url2 = requests.get("https://finance.naver.com/sise/sise_market_sum.naver?sosok=1&page=1")
html2 = BeautifulSoup(url2.text)
kosdaq_page = int(html2.find("td", class_ = "pgRR").find("a")["href"].split("=")[-1])
kosdaq_page 

36

In [66]:
from tqdm import tqdm

total2 = []
for i in tqdm(range(1,kosdaq_page+1)):        
    dic = {"menu" : "market_sum", 
       "returnUrl" : f"http://finance.naver.com/sise/sise_market_sum.naver?sosok=1&page={i}",
       "fieldIds" : items_box}
    url2 = requests.post("https://finance.naver.com/sise/field_submit.naver", data = dic)
    html2 = BeautifulSoup(url2.text)
    table2 = make_table(html2)
    total2.append(table2)
    time.sleep(1 + random.random())

100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [01:11<00:00,  1.99s/it]


In [67]:
kosdaq = pd.concat(total2, ignore_index = True)
kosdaq["소속"]  = "KOSDAQ"
kosdaq

,종목명,현재가,전일비,등락률,액면가,거래량,거래대금,전일거래량,시가,고가,...,보통주배당금,매출액증가율,영업이익증가율,외국인비율,PER,ROE,ROA,PBR,유보율,소속
0,알테오젠,395000.0,"상승 1,500",+0.38%,500.0,186054.0,72796.0,416909.0,397000.0,397000.0,...,0.0,235.08,66.85,16.01,-7181.82,-2.27,-1.43,118.32,445.0,KOSDAQ
1,에코프로비엠,139000.0,"상승 5,900",+4.43%,500.0,450208.0,61404.0,715115.0,133000.0,139100.0,...,NaN,28.81,-59.01,10.48,-62.56,-0.64,1.41,10.47,2633.7,KOSDAQ
2,HLB,88000.0,상승 300,+0.34%,500.0,314761.0,27585.0,1322836.0,87800.0,88200.0,...,0.0,-76.13,-67.45,22.72,-61.80,-32.55,-23.97,22.00,1058.0,KOSDAQ
3,에코프로,67200.0,"상승 1,800",+2.75%,100.0,499475.0,33122.0,977768.0,64500.0,67200.0,...,0.0,28.73,-51.38,20.03,-49.23,3.43,2.10,5.86,9676.2,KOSDAQ
4,레인보우로보틱스,389000.0,"상승 2,000",+0.52%,500.0,127596.0,49720.0,412248.0,394000.0,395000.0,...,NaN,12.07,-3527.67,8.35,-368.72,-0.93,-0.81,58.26,1223.0,KOSDAQ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1786,투비소프트,583.0,하락 28,-4.58%,500.0,190039.0,112.0,816914.0,611.0,629.0,...,0.0,-0.46,8.06,2.89,-0.19,-41.73,-17.86,0.20,-8.0,KOSDAQ
1787,셀리버리,116.0,하락 27,-18.88%,500.0,7268545.0,850.0,31418566.0,125.0,125.0,...,0.0,-18.55,70.75,3.78,-0.70,472.60,-59.26,-0.21,-289.3,KOSDAQ
1788,CNH,109.0,보합0,0.00%,2500.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-17.10,76.64,6.07,-0.04,-18.80,-3.89,0.36,5.8,KOSDAQ
1789,소프트센우,7060.0,상승 140,+2.02%,200.0,1186.0,8.0,204.0,7050.0,7060.0,...,NaN,NaN,NaN,0.02,-110.31,NaN,NaN,13.72,NaN,KOSDAQ


In [68]:
stock = pd.concat([kospi, kosdaq], ignore_index = True)
stock.to_excel("시가총액 - 코스피,코스닥의 모든열.xlsx", index=False)
stock

,종목명,현재가,전일비,등락률,액면가,거래량,거래대금,전일거래량,시가,고가,...,보통주배당금,매출액증가율,영업이익증가율,외국인비율,PER,ROE,ROA,PBR,유보율,소속
0,삼성전자,56400.0,하락 800,-1.40%,100.0,8375923.0,473227.0,14625181.0,57000.0,57100.0,...,1446.0,16.20,398.34,50.07,11.39,9.03,7.10,0.97,41772.8,KOSPI
1,SK하이닉스,204500.0,"상승 4,000",+2.00%,5000.0,1475810.0,298985.0,3651613.0,199000.0,206000.0,...,1200.0,-26.57,-213.52,55.84,14.27,-15.61,-8.95,2.16,1397.1,KOSPI
2,LG에너지솔루션,371500.0,"하락 2,000",-0.54%,500.0,59939.0,22291.0,140338.0,369500.0,376000.0,...,NaN,31.83,78.23,4.53,-239.68,6.36,3.91,4.30,16691.6,KOSPI
3,삼성바이오로직스,1159000.0,"하락 19,000",-1.61%,2500.0,24619.0,28593.0,108833.0,1181000.0,1187000.0,...,NaN,23.10,13.22,13.58,78.37,9.12,5.26,7.79,5432.6,KOSPI
4,현대차,206000.0,"상승 3,500",+1.73%,5000.0,289503.0,59493.0,651155.0,203500.0,207000.0,...,11400.0,14.43,53.96,37.18,4.51,13.68,4.56,0.54,6248.8,KOSPI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4096,투비소프트,583.0,하락 28,-4.58%,500.0,190039.0,112.0,816914.0,611.0,629.0,...,0.0,-0.46,8.06,2.89,-0.19,-41.73,-17.86,0.20,-8.0,KOSDAQ
4097,셀리버리,116.0,하락 27,-18.88%,500.0,7268545.0,850.0,31418566.0,125.0,125.0,...,0.0,-18.55,70.75,3.78,-0.70,472.60,-59.26,-0.21,-289.3,KOSDAQ
4098,CNH,109.0,보합0,0.00%,2500.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-17.10,76.64,6.07,-0.04,-18.80,-3.89,0.36,5.8,KOSDAQ
4099,소프트센우,7060.0,상승 140,+2.02%,200.0,1186.0,8.0,204.0,7050.0,7060.0,...,NaN,NaN,NaN,0.02,-110.31,NaN,NaN,13.72,NaN,KOSDAQ


# 동적 사이트 수집하는 법 - Selenium

In [78]:
# Selenium : 수집용, 매크로
# 앞에 배웟던것들이랑 다름 주의 

# !pip install undetected-chromedriver
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

driver = uc.Chrome()          # 새 크롬창 열림
time.sleep(3)
driver.get("https://www.naver.com")         # 네이버로 들어가라고 명령

In [81]:
search = driver.find_element(By.CLASS_NAME, "search_input")    # 검색창에 대한 요소 찾음
search.send_keys("대구은행")       # 검색할 내용 
time.sleep(1)
search.send_keys(Keys.RETURN)       # 엔터키 쳐줌 

In [84]:
# 팝업창 닫기

driver = uc.Chrome()
driver.get("https://tickets.interpark.com/goods/24015987")
time.sleep(2)
driver.find_element(By.CLASS_NAME, "popupCloseBtn.is-bottomBtn").click()  # 닫을때 click

# 셀레니움은 공백이 있으면 안됨.   공백이있으면 쩜(.)으로 바꿔주기 

In [91]:
# 이용후기 들어가는 법 

A = driver.find_element(By.CSS_SELECTOR, "ul.navList").find_elements (By.CSS_SELECTOR, "li.navItem")
for i in A:
    if "이용후기" in i.text:     # 내용보는건 .text로 동일 
        i.click()
        
# CSS_SELECTOR: 태그명, 클래스 속성 같이 적을수 있음 
# find_all 아닌 find_elements로

In [112]:
# 종합;  크롬창 열어서 인터파크 들어가, 팝업창끄고 이용후기에 들어가라
driver = uc.Chrome()
driver.get("https://tickets.interpark.com/goods/24015987")
time.sleep(1)
driver.find_element(By.CLASS_NAME, "popupCloseBtn.is-bottomBtn").click()
time.sleep(1)
A = driver.find_element(By.CSS_SELECTOR, "ul.navList").find_elements (By.CSS_SELECTOR, "li.navItem")
for i in A:
    if "이용후기" in i.text:     # 내용보는건 .text로 동일 
        i.click()

In [122]:
# 이용후기의 1페이지의 제목,내용 등을 모두 가져오는 법 

box = []
for i in driver.find_elements(By.CSS_SELECTOR, "li.bbsItem"):
    title = (i.find_element(By.CSS_SELECTOR, "strong.bbsTitleText").text.strip())   # 안에 포함된 공백 없애기 위해 strip
    content = (i.find_element(By.CSS_SELECTOR, "p.bbsText").text.strip())
    star = (i.find_element(By.CSS_SELECTOR, "div.prdStarIcon").get_attribute("data-star"))
                                                              # 찾은 부분의 속성에 접근 
    name = (i.find_element(By.CSS_SELECTOR, "span.name").text.strip())
    date = (i.find_elements(By.CSS_SELECTOR, "li.bbsItemInfoList")[1].text)
    box.append([title, content, name, star, date])

In [123]:
pd.DataFrame(box, columns = ["제목", "내용", "작성자", "별점" ,"날짜"])

,제목,내용,작성자,별점,날짜
0,실망,사람 너무너무너무 많고 혼잡했습니다.,judy_0***,1,2025.02.26
1,생각보다 별로,전시장이 어둡지만 작품에는 조명이 있어 몰입은 잘 됨 하지만 관람 동선이 따로 없어...,chorong0***,2,2025.02.24
2,이런 전시회는......,이렇게 혼잡하고 관람이 불편한 전시회는 처음입니다.\n\n그림을 제대로 볼 수도 없...,wellif***,1,2025.02.23
3,2025 최악의 전시,좁은 공간. 빡센 밀도. 꼬이는 동선.\n작품을 볼 수 없는 작품전.,gyalsd***,1,2025.02.23
4,전시장 관리가 아예 안됨,"관람 인원도 제대로 관리를 안 하는 건지, 내부가 과하게 혼잡스러움.\n어디서는 자...",sl040***,1,2025.02.23
5,반고흐 전시회 후기,전시장에 들어서서 실물 티켓 발권과 입장까지 어수선한 느낌이었습니다.\n많은 사랑을...,jseul***,4,2025.02.22
6,생각보다 실망스럽네요.,저도 다녀오니 왜 평점이 낮은지 알겠네요.\n사람 최대한 피해서 평일 오전에 다녀왔...,sylph***,3,2025.02.22
7,후기가 안좋은 이유가 있네요,전시의 질보다는\n\n운영의 미숙함이 크게 다가옵니다\n\n예술의 전당 운영능력이 ...,ewlove***,4,2025.02.21
8,아쉽네요.,"가격대비 작품 수도 그렇게 많지 않은데, 사람은 여전히 많아서 보기도 불편하고......",beatofang***,2,2025.02.21
9,천재 화가를 만나다.,빈센트 반 고흐의 전시를 보며 고통속에서도\n그림에 대한 열정을 놓치 않있던 예슬가...,youje***,5,2025.02.20


In [161]:
# 1페이지부터 마지막페이지까지 클릭해보기

box = []
while True:
    numbers = driver.find_element(By.CSS_SELECTOR, "ol.pageNumWrap").find_elements(By.CSS_SELECTOR, "li")
    
    for i in numbers:
        i.click()             # 1~ 10페이지까지 돌아라 
        time.sleep(1)
        
    try:
        driver.find_element(By.CSS_SELECTOR, "a.pageNextBtn.pageArrow").click()     # 다음페이지 클릭
        time.sleep(1)
    except: 
        break             # 혹시 에러나면 나가라 

In [163]:
# 이용후기의 모든 페이지의 제목,내용 등을 모두 가져오는 법 

box = []
while True:
    numbers = driver.find_element(By.CSS_SELECTOR, "ol.pageNumWrap").find_elements(By.CSS_SELECTOR, "li")
    
    for n in numbers:
        n.click()             # 1~ 10페이지까지 돌아라 
        time.sleep(1)

        for i in driver.find_elements(By.CSS_SELECTOR, "li.bbsItem"):
            title = (i.find_element(By.CSS_SELECTOR, "strong.bbsTitleText").text.strip())   # 안에 포함된 공백 없애기 위해 strip
            content = (i.find_element(By.CSS_SELECTOR, "p.bbsText").text.strip())
            star = (i.find_element(By.CSS_SELECTOR, "div.prdStarIcon").get_attribute("data-star"))
            name = (i.find_element(By.CSS_SELECTOR, "span.name").text.strip())
            date = (i.find_elements(By.CSS_SELECTOR, "li.bbsItemInfoList")[1].text)
            box.append([title, content, name, star, date])
        
    try:
        driver.find_element(By.CSS_SELECTOR, "a.pageNextBtn.pageArrow").click()     # 다음페이지 클릭
        time.sleep(1)
    except: 
        break             # 혹시 에러나면 나가라 

In [164]:
result = pd.DataFrame(box, columns = ["제목", "내용", "작성자", "별점" ,"날짜"])
result.to_excel("인터파크.xlsx", index = False)
result 

,제목,내용,작성자,별점,날짜
0,실망,사람 너무너무너무 많고 혼잡했습니다.,judy_0***,1,2025.02.26
1,생각보다 별로,전시장이 어둡지만 작품에는 조명이 있어 몰입은 잘 됨 하지만 관람 동선이 따로 없어...,chorong0***,2,2025.02.24
2,이런 전시회는......,이렇게 혼잡하고 관람이 불편한 전시회는 처음입니다.\n\n그림을 제대로 볼 수도 없...,wellif***,1,2025.02.23
3,2025 최악의 전시,좁은 공간. 빡센 밀도. 꼬이는 동선.\n작품을 볼 수 없는 작품전.,gyalsd***,1,2025.02.23
4,전시장 관리가 아예 안됨,"관람 인원도 제대로 관리를 안 하는 건지, 내부가 과하게 혼잡스러움.\n어디서는 자...",sl040***,1,2025.02.23
...,...,...,...,...,...
176,전시회,기대만큼 좋은 전시회!,cji***,5,2024.12.03
177,"정말 실망이네요,,,","추운 겨울에 제가 가장 좋아하는 고흐의 작품을 기대를 품고 보러 갔는데,\n얼리버드...",musics***,1,2024.12.03
178,예술의전당 전시회 실망하긴 처음 이네요...ㅜ,사상최대라해서 기대하고갔는데...음..\n어수선한 전시회장 분위기와 현장에서 발권해...,Psh***,1,2024.12.02
179,사람이 이렇게나 많은데 무조건 현장종이발권,언제적 아날로그 입장인거죠...?? 그리고 고흐 제일 유명한 그림들은 없습니다 이거...,effy1***,2,2024.12.02
